In [1]:
# default_exp capture

# Capture

> Wrapper class and example code for getting images from the OpenHSI using a ximea detetor (with IMX252 sensor, e.g. MX031CG-SY).Wrapper class and example code for getting images from the OpenHSI using a ximea detetor (with IMX252 sensor, e.g. [MX031CG-SY](https://www.ximea.com/en/products/xilab-application-specific-custom-oem/embedded-vision-and-multi-camera-setup-xix/sony-imx252-fast-color-industrial-camera)).

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export
import numpy as np
import matplotlib.pyplot as plt
from ximea import xiapi

In [11]:
#export
class OpenHSI(object):
    """XIMEA camera class to take images"""

    def __init__(self, serialnumber:str = None, xbinwidth:int = 896, xbinoffset:int = 528, 
                exposure_ms:int = 1000, gain:int = 0):
        """Init the camera"""
        self.xicam = xiapi.Camera()
        self.xicam.open_device_by_SN(serialnumber) if serialnumber else self.xicam.open_device()

        print(f'Connected to device {self.xicam.get_device_sn()}')

        self.xbinwidth  = xbinwidth
        self.xbinoffset = xbinoffset
        self.exposure   = exposure_ms # ms?
        self.gain       = 0

        self.xicam.set_width(self.xbinwidth)
        self.xicam.set_offsetX(self.xbinoffset)
        self.xicam.set_exposure_direct(1000*self.exposure)
        self.xicam.set_gain_direct(self.gain)

        # What other settings are there?, can we make these *args, **kwargs? and check 
        # them using getattr()
        self.xicam.set_imgdataformat("XI_RAW16")
        self.xicam.set_output_bit_depth("XI_BPP_12")
        self.xicam.enable_output_bit_packing()
        self.xicam.disable_aeag() # what is this?
        
        self.xicam.set_binning_vertical(2)
        self.xicam.set_binning_vertical_mode("XI_BIN_MODE_SUM")

        self.rows, self.cols = self.xicam.get_height(), self.xicam.get_width()

        self.img = xiapi.Image()
        self.xicam.start_acquisition()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.xicam.stop_acquisition()
        self.xicam.close_device()

    @property
    def exposure(self) -> float: # or int?
        """Current exposure in ms."""
        return self.xicam.get_exposure() / 1000

    @exposure.setter
    def exposure(self, val):
        """Update exposure in ms."""
        self.xicam.set_exposure_direct(val * 1000)

    @property
    def gain(self) -> float: # or int?
        """Current exposure property in dB from 0 to 24 dB."""
        return self.xicam.get_gain()

    @gain.setter
    def gain(self, val):
        """Update gain in dB."""
        self.xicam.set_gain_direct(val)

    # possibly put in getter and setter methods for xbinwidth and xbinoffset

    def get_img(self, n:int = 1, show:bool = True) -> np.ndarray:
        """Take `n` images with option to `show` first image taken."""

        data = np.zeros((self.cols,self.rows,n), dtype = np.uint16)
        for i in range(n):
            self.xicam.get_image(self.img)
            data[...,i] = np.rot90(self.img.get_image_data_numpy(), -1)
        
        if show and n > 0:
            plt.imshow(data[...,0])
            plt.xlabel('Wavelength (nm)')
            plt.ylabel('Line pixels')
            plt.show()

        return data




The `OpenHSI` class contains getters and setters to return and modify the `exposure` (in milliseconds) and `gain`. 

In [12]:
show_doc(OpenHSI.get_img)

<h4 id="OpenHSI.get_img" class="doc_header"><code>OpenHSI.get_img</code><a href="__main__.py#L67" class="source_link" style="float:right">[source]</a></h4>

> <code>OpenHSI.get_img</code>(**`n`**:`int`=*`1`*, **`show`**:`bool`=*`True`*)

Take `n` images with option to `show` first image taken.

In [13]:
#export
def take_show():
    """Grab a single image using `get_img` and plot it"""
    with OpenHSI(xbinwidth=896,xbinoffset=528,exposure_ms=1000,gain=0) as cam:
        img = cam.start().get_img()
        plt.imshow(img)
        plt.xlabel('Wavelength (nm)')
        plt.ylabel('Line pixels')
        plt.show()

For convience, a function is provided that initialises the camera, takes an image, and display it. 